In [2]:
!pip install SAGEDbias==0.0.11

from saged import SAGEDData, SourceFinder, Scraper
import pandas as pd
import re
from tqdm import tqdm

domains_and_categories = {
    "nationalities": ["British people", "American people", "Indian people"],
    "religion": ["Christianity", "Islam", "Hinduism", "Atheism"],
    "gender": ["Male", "Female", "Non-binary", "Transgender"],
    "lgbtq": ["Gay", "Lesbian", "Bisexual", "Transgender"],
    "languages": ["English speakers", "Spanish speakers", "Mandarin speakers"],
    "age_groups": ["Children", "Teenagers", "Adults", "Seniors"]
}


rows = []


for domain, categories in tqdm(domains_and_categories.items(), desc="Processing Domains"):
    for category in categories:
        print(f"Processing: {domain} - {category}")

        try:

            keywords_data = SAGEDData.create_data(domain, category, "keywords")

            keywords_to_add = {
                "nationalities": ["Brit", "UK", "USA", "Indian"],
                "religion": ["Faith", "Belief", "God", "Spirituality"],
                "gender": ["He", "She", "They", "Identity"],
                "lgbtq": ["Pride", "Equality", "Love", "Freedom"],
                "languages": ["Speaking", "Fluent", "Native", "Learning"],
                "age_groups": ["Youth", "Adulting", "Mature", "Senior Citizen"],
            }.get(domain, [])

            for keyword in keywords_to_add:
                keywords_data.add(keyword=keyword)
            source_finder = SourceFinder(keywords_data)
            wiki_sources = source_finder.find_scrape_urls_on_wiki(top_n=2, scrape_backlinks=2)

            if not wiki_sources.data:
                print(f"No wiki sources found for category: {category}")
                continue

            scraper = Scraper(wiki_sources)
            scraper.scrape_in_page_for_wiki_with_buffer_files()
            scraped_sentences_data = scraper.scraped_sentence_to_saged_data()

            if not scraped_sentences_data.data:
                print(f"No data found for category: {category}")
                continue
            if 'keywords' not in scraped_sentences_data.data[0]:
                print(f"No keywords data for category: {category}")
                continue

            for keyword, keyword_data in scraped_sentences_data.data[0]['keywords'].items():
                if 'scraped_sentences' not in keyword_data:
                    print(f"No scraped sentences for keyword: {keyword}")
                    continue

                scraped_sentences = keyword_data.get('scraped_sentences', [])
                for sentence, _ in scraped_sentences:
                    stereotype_type = domain.capitalize()
                    category_label = category
                    label = "Biased" if domain in ["nationalities", "gender", "lgbtq", "age_groups"] else "Neutral"

                    marked_text = sentence
                    for word in keywords_to_add:
                        marked_text = re.sub(r'\b' + re.escape(word) + r'\b', f'==={word}===', marked_text, flags=re.IGNORECASE)

                    rows.append({
                        'stereotype_type': stereotype_type,
                        'text': sentence,
                        'text_with_marker': marked_text,
                        'category': category_label,
                        'data_source': 'scraped_wikipedia',
                        'label': label
                    })

        except Exception as e:
            print(f"Error processing {domain} - {category}: {e}")
            continue

df = pd.DataFrame(rows)
output_file = "scraped_bias_dataset.csv"
df.to_csv(output_file, index=False)
print(f"Scraped and processed data saved to '{output_file}'")


Processing Domains:   0%|          | 0/6 [00:00<?, ?it/s]

Processing: nationalities - British people
Searching Wikipedia for topic: British people
Found Wikipedia page: British people
Searching similar forelinks for British people



Depth 1/1: 100%|██████████| 2/2 [00:00<00:00,  7.93it/s]


Searching similar backlinks for British people



Depth 1/1: 100%|██████████| 2/2 [00:00<00:00,  7.00it/s]

Scraping through URL:   0%|          | 0/3 [00:00<?, ?url/s]

Scraping in page:   0%|          | 0/4 [00:00<?, ?keyword/s]

Scraping in page:  25%|██▌       | 1/4 [00:00<00:02,  1.04keyword/s]

Scraping in page:  50%|█████     | 2/4 [00:01<00:01,  1.16keyword/s]

Scraping in page:  75%|███████▌  | 3/4 [00:02<00:00,  1.21keyword/s]

Scraping in page: 100%|██████████| 4/4 [00:03<00:00,  1.05keyword/s]

Scraping through URL:  33%|███▎      | 1/3 [00:03<00:07,  3.81s/url]

Scraping in page:   0%|          | 0/4 [00:00<?, ?keyword/s]

Scraping in page:  25%|██▌       | 1/4 [00:00<00:00,  3.10keyword/s]

Scraping in page:  50%|█████     | 2/4 [00:00<00:00,  2.97keyword/s]

Scraping in page:  75%|███████▌  | 3/4 [00:00<00:00,  3.02keyword/s]

Scraping in page: 100%|██████████| 4/4 [00:01<00:00,  3.01keyword/s]

Scraping through URL:  67%|██████▋   | 2/3 [00:05<00:02,  2.36s/url]

Scraping in page:   0%|          | 0/4 [00:00<?, ?keywo

...

Processing: age_groups - Seniors
Searching Wikipedia for topic: Seniors
Found Wikipedia page: Senior
Searching similar forelinks for Seniors



Depth 1/1: 100%|██████████| 2/2 [00:00<00:00,  7.90it/s]


Searching similar backlinks for Seniors



Depth 1/1: 100%|██████████| 2/2 [00:00<00:00,  8.51it/s]

Scraping through URL:   0%|          | 0/3 [00:00<?, ?url/s]

Scraping in page:   0%|          | 0/4 [00:00<?, ?keyword/s]

Scraping in page:  25%|██▌       | 1/4 [00:00<00:00,  3.24keyword/s]

Scraping in page:  50%|█████     | 2/4 [00:00<00:00,  4.21keyword/s]

Scraping in page:  75%|███████▌  | 3/4 [00:00<00:00,  4.66keyword/s]

Scraping in page: 100%|██████████| 4/4 [00:00<00:00,  4.55keyword/s]

Scraping through URL:  33%|███▎      | 1/3 [00:00<00:01,  1.12url/s]

Scraping in page:   0%|          | 0/4 [00:00<?, ?keyword/s]

Scraping in page:  25%|██▌       | 1/4 [00:00<00:00,  4.28keyword/s]

Scraping in page:  50%|█████     | 2/4 [00:00<00:00,  4.19keyword/s]

Scraping in page:  75%|███████▌  | 3/4 [00:00<00:00,  4.19keyword/s]

Scraping in page: 100%|██████████| 4/4 [00:00<00:00,  4.17keyword/s]

Scraping through URL:  67%|██████▋   | 2/3 [00:01<00:00,  1.06url/s]

Scraping in page:   0%|          | 0/4 [00:00<?, ?keywo

Error processing age_groups - Seniors: 'scraped_sentences'
Scraped and processed data saved to 'scraped_bias_dataset.csv'


In [3]:
!pip install SAGEDbias==0.0.11

In [2]:
!pip install SAGEDbias==0.0.11
!pip install transformers
!pip install docling
!pip install ollama

import os
import pandas as pd
from saged import KeywordFinder, SourceFinder, Scraper, SAGEDData
from transformers import pipeline
import re
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor
import time
from functools import wraps

# Step 1: Define race-related domains and categories
domains_and_categories = {
    "race_ethnicity": [
        "White people",
        "Black people",
        "Asian people",
        "Hispanic people",
        "Native American people",
        "Middle Eastern people",
        "African people",
        "European people",
        "Mixed race people"
    ]
}

# Race list for generating and highlighting keywords
race_list = [
    "White", "Black", "Asian", "Hispanic",
    "Native American", "Middle Eastern",
    "African", "European", "Mixed"
]

# Step 2: Define LLM-based keyword generation class
class HuggingFaceChatPipeline:
    def __init__(self, model_name="Qwen/Qwen2.5-1.5B-Instruct"):
        self.chat_pipeline = pipeline(
            "text-generation",
            model=model_name,
            tokenizer=model_name,
            device_map="auto",
            torch_dtype="auto"
        )

    def invoke(self, user_prompt, system_prompt="You are a helpful assistant."):
        prompt = f"{system_prompt}\n\nUser: {user_prompt}\n\nAssistant:"
        response = self.chat_pipeline(
            prompt,
            max_length=512,
            num_return_sequences=1,
            pad_token_id=self.chat_pipeline.tokenizer.eos_token_id
        )[0]["generated_text"]
        response_cleaned = response.replace(prompt, "").strip()
        return response_cleaned

# Step 3: Retry Logic for Resilient Scraping
def retry(max_attempts, delay):
    def decorator(func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            for attempt in range(max_attempts):
                try:
                    return func(*args, **kwargs)
                except Exception as e:
                    if attempt < max_attempts - 1:
                        time.sleep(delay)
                    else:
                        raise e
        return wrapper
    return decorator

@retry(max_attempts=3, delay=5)
def safe_scrape(scraper):
    scraper.scrape_in_page_for_wiki_with_buffer_files()

# Step 4: Processing Function
def process_category(category, domain, race_list, model):
    rows = []
    try:
        keyword_finder = KeywordFinder(category, domain)

        # Generate Keywords
        keyword_finder.find_keywords_by_embedding_on_wiki(n_keywords=3)
        keywords_data_embeddings = keyword_finder.keywords_to_saged_data()

        keyword_finder.find_keywords_by_llm_inquiries(
            generation_function=model.invoke, n_keywords=3, n_run=2
        )
        keywords_data_llm = keyword_finder.keywords_to_saged_data()

        # Combine keywords
        keywords_data = keywords_data_embeddings.merge(merge_list=[keywords_data_llm])

        # Scrape Sources
        source_finder = SourceFinder(keywords_data)
        wiki_sources = source_finder.find_scrape_urls_on_wiki(top_n=2, scrape_backlinks=0)

        if not wiki_sources.data:
            print(f"No wiki sources found for category: {category}")
            return rows

        scraper = Scraper(wiki_sources)
        safe_scrape(scraper)
        scraped_sentences_data = scraper.scraped_sentence_to_saged_data()

        # Process Scraped Sentences
        for keyword, keyword_data in scraped_sentences_data.data[0]["keywords"].items():
            scraped_sentences = keyword_data.get("scraped_sentences", [])
            for sentence, _ in scraped_sentences:
                marked_text = sentence
                for word in race_list:
                    marked_text = re.sub(rf"\b{re.escape(word)}\b", f"==={word}===", marked_text, flags=re.IGNORECASE)

                rows.append({
                    "stereotype_type": domain.capitalize(),
                    "text": sentence,
                    "text_with_marker": marked_text,
                    "category": category,
                    "data_source": "scraped_wikipedia",
                    "label": "Biased"
                })
    except Exception as e:
        print(f"Error processing category {category}: {e}")
    return rows

# Step 5: Main Script with Parallel Processing
def main():
    model = HuggingFaceChatPipeline(model_name="Qwen/Qwen2.5-1.5B-Instruct")
    all_rows = []

    with ThreadPoolExecutor(max_workers=4) as executor:  # Adjust max_workers based on your CPU
        futures = [
            executor.submit(process_category, category, domain, race_list, model)
            for domain, categories in domains_and_categories.items()
            for category in categories
        ]
        for future in tqdm(futures, desc="Processing categories"):
            all_rows.extend(future.result())

    # Save results to CSV
    output_file = "optimized_race_bias_dataset.csv"
    df = pd.DataFrame(all_rows)
    df.to_csv(output_file, index=False)
    print(f"Dataset saved to '{output_file}'.")

if __name__ == "__main__":
    main()


Initiating the embedding model...Initiating the embedding model...
Initiating the embedding model...

Initiating the embedding model...


Processing categories:   0%|          | 0/9 [00:00<?, ?it/s]

Batches:   0%|          | 0/13 [00:00<?, ?it/s]

Batches:   0%|          | 0/64 [00:00<?, ?it/s]

Batches:   0%|          | 0/89 [00:00<?, ?it/s]

...

Error processing category European people: SAGEDData.merge() missing 1 required positional argument: 'domain'





finding keywords by LLM: 100%|██████████| 2/2 [00:42<00:00, 21.31s/run]

Response: ['Mixed heritage', 'Multiple racial identities', 'Inter-marriage or cross-cultural relationships', 'Cultural blending and syncretism', 'Complex family histories', 'Challenges related to identity formation', 'Opportunities for cultural exchange and understanding', 'Intersectionality within social justice contexts']
final set of keywords:
['Octo-racial', 'Inter-marriage or cross-cultural relationships', 'Quadri-racial', 'ethnic', 'Biracial', 'Cultural blending and syncretism', 'Tri-racial', 'Multiple racial identities', 'Challenges related to identity formation', 'Multiracial', 'Transracial', 'Septi-racial', 'Intersectionality within social justice contexts', 'Opportunities for cultural exchange and understanding', 'Quinti-racial', 'mixed', 'Complex family histories', 'Heteroracial', 'race']



Processing categories: 100%|██████████| 9/9 [06:02<00:00, 40.23s/it]

Error processing category Mixed race people: SAGEDData.merge() missing 1 required positional argument: 'domain'
Dataset saved to 'optimized_race_bias_dataset.csv'.
